# DATA COLLECTION
---

Nhiệm vụ phần này: từ danh sách các đường dẫn phim của trang web `Just Watch`, thực hiện cào thông tin **imdbId** trong đường dẫn và dùng **imdbId** đó để cào thông tin của phim đó trong trang web `OMDB` bằng api.

## 1. IMPORT THƯ VIỆN CẦN THIẾT

In [9]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re

## 2. HÀM CÀO THÔNG TIN IMDBID TRONG TRANG `JUST WATCH`

In [8]:
links = []

with open('../Data/links.txt', 'r') as f:
    links = [line.strip() for line in f]

Dùng selenium thoi vì request k được

In [13]:
# chia 3 ra nha mina hong no k cho minh lay :D
# từ từ để ngâm cú chứ nhìn này trông cx hơi ngu :v
member_part = {
    "Trinh": {
        "part": 1,
        "api_key": None, # đk api xong áp vô nè
        "start": 0,
        "end": 628
    },

    "Thi": {
        "part": 2,
        "api_key": None, # đk api xong áp vô nè
        "start": 808,
        "end": 1258
    },

    "MinhUyen": {
        "part": 3,
        "api_key": 'd4c83dec',
        "start": 1259,
        "end": 1887
    } 
}

In [14]:
# Điền tên mn vào đây theo từ điển nè
info = member_part["MinhUyen"]

part = info["part"]
apiKey = info["api_key"]
start = info["start"]
end = info["end"]

print(part)
print(apiKey)
print(start)
print(end)

3
d4c83dec
1259
1887


In [26]:
def collect_imdbID_title(links, start, end):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    imdbIds_titles = []
    for url in links[start: end]:
        print(f"[LOG]: parse imdbid of url {url}")
        driver.get(url)
        page_source = driver.page_source

        imdb_id_match = re.search(r'"imdbId":"(tt\d+)"', page_source)
        if imdb_id_match:
            imdb_id = imdb_id_match.group(1)
            imdbIds_titles.append(imdb_id)
            print("IMDB ID:", imdb_id)
        else:
            # print("Không tìm thấy IMDb ID.")
            movie_title_meta = driver.find_element(By.XPATH, "//meta[@property='og:title']")
            movie_title = movie_title_meta.get_attribute("content").split(" streaming")[0] if movie_title_meta else "Null"
            imdbIds_titles.append(movie_title)
            print(movie_title)

        time.sleep(3)

    driver.quit()

    return imdbIds_titles

In [27]:
imdbIds_titles = collect_imdbID_title(links, start=start, end=end)
imdbIds_titles

[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/goon-squad
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-phantom-warrior
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/beyond-the-lake


['tt21625056', 'tt13830348', 'tt21608540']

## 3. HÀM LẤY DỮ LIỆU TỪ API CỦA TRANG `OMDB`

In [ ]:
def collectMovieData(imdbIds, key = 0):
    title = []
    rated = []
    released = []
    runtime = []
    genre = []
    metaScore = []
    rating = []
    vote = []
    boxCffice = []

    for id in imdbIds:
        if id is not "Null":
            print(f"[LOG]: Collecting data of IMDbId {id}")

            url = f'https://www.omdbapi.com/?i={id}&apikey={key}'
            trying = 1

            while True:    
                respond = requests.get(url)
                if respond.status_code == 200:
                    data = respond.json()
                    title.append(data['Title'])
                    rated.append(data['Rated'])
                    released.append(data['Released'])
                    runtime.append(data['Runtime'])
                    genre.append(data['Genre'])
                    metaScore.append(data['Metascore'])
                    rating.append(data['imdbRating'])
                    vote.append(data['imdbVotes'])
                    boxCffice.append(data['BoxOffice'])
                    break
                    
                else:
                    trying += 1
                    if trying > 3:
                        print(f"Request fail after 3 retries with code {respond.status_code}")

                    time.sleep(1)

    movieDataframe = pd.DataFrame({
            "Title": title,
            "Rated": rated,
            "Released": released,
            "Runtime": runtime,
            "Genre": genre,
            "Metascore": metaScore,
            "IMDb Rating": rating,
            "IMDb Votes": vote,
            "Domestic Revenue": boxCffice

        })

    return movieDataframe

In [ ]:
df1 = collectMovieData(imdbIds_titles, key=apiKey)
df1

In [22]:
print(os.path.join("./Data/", f"raw_movie_data_{str(part)}.csv"))

./Data/raw_movie_data_3.csv


In [ ]:
# Lưu file đây nhé
file_name = os.path.join("./Data/", f"raw_movie_data_{str(part)}.csv")
df1.to_csv(file_name)